In [ ]:
import pandas as pd
import sqlite3
import os


Checking column names and looking for join potential

In [ ]:
# Checking column names and looking for join potential
df_Recruiting_Ranking_2022 = pd.read_csv('Recruiting_Ranking_2022.csv')
print(df_Recruiting_Ranking_2022.columns)

In [ ]:
#Combining Highschool recruiting ranking csv for years 202019-2023 ( there should be 4 people per rank)
# Added more years to appease a friend and get specific data to compare

In [ ]:


files_to_combine = [
    'Recruiting_Ranking_2015.csv',
    'Recruiting_Ranking_2016.csv',
    'Recruiting_Ranking_2017.csv',
    'Recruiting_Ranking_2018.csv',
    'Recruiting_Ranking_2019.csv',
    'Recruiting_Ranking_2020.csv',
    'Recruiting_Ranking_2021.csv',
    'Recruiting_Ranking_2022.csv',
    'Recruiting_Ranking_2019.csv',
]

Recruiting_Ranking = []

for file in files_to_combine:
    College = pd.read_csv(file)
    Recruiting_Ranking.append(College)

Recruiting_Rank_Combined = pd.concat(Recruiting_Ranking, ignore_index=True)

print(Recruiting_Rank_Combined.head(10))

In [ ]:
# Combining NFL Draft results from 2022-2025
# Added more draft years to appease a friend and compare specific data
files_to_combine_draft = [
    'NFL_Draft_2019.csv',
    'NFL_Draft_2020.csv',
    'NFL_Draft_2021.csv',
    'NFL_Draft_2022.csv',
    'NFL_Draft_2023.csv',
    'NFL_Draft_2024.csv',
    'NFL_Draft_2025.csv'
]
NFL_Draft = []
for file in files_to_combine_draft:
    NFL = pd.read_csv(file)
    NFL_Draft.append(NFL)

NFL_Draft_Combined = pd.concat(NFL_Draft, ignore_index=True)

print(NFL_Draft_Combined.head(10))

Looking at results

In [ ]:
# looking at results
print(NFL_Draft_Combined.head(15))

In [ ]:
#Looking at shape to see if requirements were met
NFL_Draft_Combined.shape

In [ ]:
Recruiting_Rank_Combined.columns

In [ ]:
# Renaming columns for consistency as this will be my key in SQL
Recruiting_Rank_Combined.rename(columns={'AthleteId': 'CollegeAthleteId'}, inplace=True)

In [ ]:
Recruiting_Rank_Combined.columns

In [ ]:
#deleted columns 
Recruiting_Rank_Combined.drop(['HometownInfo FipsCode', 'RecruitType'], axis=1, inplace=True)



In [ ]:
#chekcing my work to make sure the columns were deleted
Recruiting_Rank_Combined.columns

In [ ]:
Recruiting_Rank_Combined.shape

In [ ]:
Recruiting_Rank_Combined.isnull().sum()
print(Recruiting_Rank_Combined.isnull().sum())

In [ ]:
Recruiting_Rank_Combined[Recruiting_Rank_Combined["CommittedTo"].isnull()]

In [ ]:
Recruiting_Rank_Combined[Recruiting_Rank_Combined["Name"] == "Zach Evans"]

In [ ]:
NFL_Draft_Combined[NFL_Draft_Combined["Name"] == "Zach Evans"]

In [ ]:
#checking for null values in CollegeAthleteId
Recruiting_Rank_Combined[Recruiting_Rank_Combined["CollegeAthleteId"].isnull()]

In [ ]:
NFL_Draft_Combined[NFL_Draft_Combined["CollegeAthleteId"].isnull()]

In [ ]:
#Creating SQLite database and saving the combined Recruiting Ranking dataframe
conn= sqlite3.connect('Football.db')

Recruiting_Rank_Combined.to_sql("Recruiting_Ranking_Combined", conn, if_exists='replace', index=False)
NFL_Draft_Combined.to_sql("NFL_Draft_Combined", conn, if_exists='replace', index=False)

In [ ]:
# Function to create a SQLite connection (1)
def sql_conn():
    return sqlite3.connect('Football.db')

In [ ]:
with sql_conn() as conn:
    df = pd.read_sql_query("""
        SELECT d.CollegeAthleteId
        FROM Recruiting_Ranking_Combined r
        RIGHT JOIN NFL_Draft_Combined d
        ON d.CollegeAthleteId = r.CollegeAthleteId;
    """, conn)

df.columns


In [ ]:
# Added latitude and longitude to the change in rating dataframe and realized I needed to change the column names to avoid issues with spaces

NFL_Draft_Combined= NFL_Draft_Combined.rename(columns={
    'HometownInfo Latitude': 'HometownInfoLatitude',
    'HometownInfo Longitude': 'HometownInfoLongitude'
})
NFL_Draft_Combined.columns

In [ ]:
    # There is something wrong here. I out of 1,000 lines that just over 50% can be joined 

with sql_conn() as conn:
    change_in_rating_df = pd.read_sql_query("""
        SELECT d.CollegeAthleteId, 
               d.Name,
               d.Position,
               d.CollegeTeam AS College, 
               r.Rating AS HighSchoolRank,
               d.NflTeam,
               d.Year AS DraftYear,
               d.Overall as Overall_Pick, 
               d.PreDraftGrade AS NFLPreDraftGrade,
               (d.PreDraftGrade / 100 - r.Rating) AS ChangeInRating,
               d."HometownInfo Latitude" AS HometownInfoLatitude,
               d."HometownInfo Longitude" AS HometownInfoLongitude
        FROM NFL_Draft_Combined d 
        LEFT JOIN Recruiting_Ranking_Combined r
        ON d.CollegeAthleteId = r.CollegeAthleteId;
    """, conn)

print(change_in_rating_df.head(15))
print(change_in_rating_df.count)



In [ ]:

Recruiting_Rank_Combined.shape


In [ ]:
    # Checking for null vallues in NFL draft key column
with sql_conn() as conn:
    null_df = pd.read_sql_query("""
        SELECT CollegeAthleteId
        FROM NFL_Draft_Combined n
        WHERE n.CollegeAthleteId IS NULL;
    """, conn)

print(null_df)

Changing column PreDraftGrade to be a percentage for finding the change in rating from high school to draft 
Note to self - College rating will be 4 decimal places, NFL predraft rating will be 2 decimal places

In [ ]:


NFL_Draft_Combined['PreDraftGrade']= NFL_Draft_Combined['PreDraftGrade']/100

print(NFL_Draft_Combined.head())

Creating a new df with new column for the change in rating from high school to draft.
Going to see if I can us this new df to see which postions have the most change in rating,

In [ ]:


with sql_conn() as conn:
    change_in_rating_df = pd.read_sql_query("""
        SELECT d.CollegeAthleteId, 
               d.Name,
               d.Position,
               d.CollegeTeam AS College, 
               r.Rating AS HighSchoolRank,
               d.NflTeam,
               d.Year AS DraftYear,
               d.[HometownInfo Latitude] AS HometownInfoLatitude,
               d.[HometownInfo Longitude] AS HometownInfoLongitude,
               d.Overall as Overall_Pick, 
               d.PreDraftGrade AS NFLPreDraftGrade,
               (d.PreDraftGrade / 100 - r.Rating) AS ChangeInRating             
        FROM NFL_Draft_Combined d 
        LEFT JOIN Recruiting_Ranking_Combined r
        ON d.CollegeAthleteId = r.CollegeAthleteId;
    """, conn)

print(change_in_rating_df.head(15))

In [ ]:
change_in_rating_df.drop_duplicates(subset=['CollegeAthleteId'], inplace=True)

In [ ]:
# Saving the  to a new table in the SQLite Table
with sql_conn() as conn:
    change_in_rating_df.to_sql("Change_In_Rating", conn, if_exists='replace', index=False)

In [ ]:
with sql_conn() as conn:
    df = pd.read_sql_query("""
        SELECT *
        FROM Change_In_Rating
        LIMIT 15;
    """, conn)

print(df)

In [ ]:
#Look at how many rows I have in the Change_In_Rating table as it should be the same as NFL Draft_Combined

with sql_conn() as conn:
    count_df = pd.read_sql_query("""
        SELECT Count(*) 
        FROM Change_In_Rating;
    """, conn)
print(count_df)

In [ ]:
print(change_in_rating_df)

In [ ]:
with sql_conn() as conn:
    Recruiting_Rank_Combined_Csv = pd.read_sql_query("""
        SELECT *
        FROM Recruiting_Ranking_Combined
        """, conn)
Recruiting_Rank_Combined_Csv.to_csv('Recruiting_Rank_Combined.csv', index=False)

In [ ]:

with sql_conn() as conn:
    NFL_Draft= pd.read_sql_query("""
        SELECT *
        FROM NFL_Draft_Combined
        """, conn)
NFL_Draft.to_csv('NFL_Draft.csv', index=False)

Making a function to return the information on a specific player that was drafted 

In [ ]:


def get_player_info(Name):
    player_info_query = f"""
        SELECT *
        FROM Change_In_Rating
        WHERE LOWER(TRIM(Name)) 
        LIKE LOWER(?)
    """
    params= f"%{Name.strip()}%"

    with sql_conn() as conn:
        return pd.read_sql_query(player_info_query, conn, params=(params,))

In [ ]:
#Checking accuracy of function

get_player_info('hardman')

In [ ]:
# Need to add 2018 and possibly 2017 Recruiting data to fill in the gaps for the NFL Draft data
# Unltimitely added 2015-2018 Recruiting data and 2019-2021 Draft data after speaking with a friend who wanted to see the data on a specific player

Creating a function to return who each NFL team drated 

In [ ]:


def get_NflTeam_Draft_results(NflTeam, DraftYear):
    team_draft_info_query = f"""
        SELECT *
        FROM Change_IN_Rating
        WHERE LOWER(TRIM(NflTeam))
        LIKE LOWER(?)
        """
    params = [f"%{NflTeam.strip()}%"]

    if DraftYear:
        team_draft_info_query += " AND DraftYear = ?"
        params.append(DraftYear)

    with sql_conn() as conn:
        return pd.read_sql_query(team_draft_info_query, conn, params=params)

In [ ]:
get_NflTeam_Draft_results('New Orleans', 2023
 )

Creating a function to search by College to see how many players were drafted from a specific college 

In [ ]:


def get_college_info(College,DraftYear):
    college_info_query = f"""
        SELECT *
        FROM Change_IN_Rating
        WHERE LOWER(TRIM(College))
        LIKE LOWER(?)
        """
    params = [f"%{College.strip()}%"]

    if DraftYear:
        college_info_query += " AND DraftYear = ?"
        params.append(DraftYear)

    with sql_conn() as conn:
        return pd.read_sql_query(college_info_query, conn, params=params)

In [ ]:
get_college_info('Lsu', 2023)

In [ ]:
# Get draft results by year (function 5)

def get_draft_results_by_year(DraftYear):
    draft_year_query = """
        SELECT *
        FROM Change_IN_Rating
        WHERE DraftYear = ?
    """
    with sql_conn() as conn:
        return pd.read_sql_query(draft_year_query, conn, params= (DraftYear,))

In [ ]:
get_draft_results_by_year(2023)

In [ ]:
get_draft_results_by_year(2024).tail(70)

In [ ]:
def inspect_null_drafts(DraftYear):
    null_draftyear_values = """
        SELECT *
        FROM Change_IN_Rating
        WHERE DraftYear = ? AND (Name IS NULL OR CollegeAthleteId IS NULL)
    """
    with sql_conn() as conn:
        return pd.read_sql_query(null_draftyear_values, conn, params=(DraftYear,))

In [ ]:
inspect_null_drafts(2023)

In [ ]:
#Creating a new file from Football.db because I found out that I cannot directly load a .db into my Tableau version

with sql_conn() as conn:
    Change_in_Rating= pd.read_sql_query("""
        SELECT *
        FROM Change_In_Rating """, conn)

Change_in_Rating.to_csv('Rating_Change.csv', index=False)